<a href="https://colab.research.google.com/github/chengwen-zheng/openAi-lab/blob/main/langchain_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [5]:

%pip install openai; 
%pip install langchain;
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

from googleapiclient.discovery import build
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
import json
import os
import openai

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()

# Build the Drive API client
drive_service = build('drive', 'v3')
file_id = '14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_'
file = drive_service.files().get(fileId=file_id).execute()
content = drive_service.files().get_media(fileId=file_id).execute()

# Read the env config.json. set env for OPENAI_API_KEY
with io.BytesIO(content) as f:
    config = json.load(f)
os.environ["OPENAI_API_KEY"] = config["api_key"];
openai.api_key = os.environ["OPENAI_API_KEY"] ;

# 选择题

In [3]:
import openai, os
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain


choicellm = ChatOpenAI(max_tokens=2048, temperature=0.5)
multiple_choice = """
请针对 >>> 和 <<< 中间的用户问题，选择一个合适的工具去回答她的问题。只要用A、B、C的选项字母告诉我答案。
如果你觉得都不合适，就选D。

>>>{question}<<<

我们有的工具包括：
A. 一个能够查询商品信息，为用户进行商品导购的工具。
B. 一个能够查询订单信息，获得最新的订单情况的工具
C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具
D. 都不合适
"""
multiple_choice_prompt = PromptTemplate(template=multiple_choice, input_variables=["question"])
choice_chain = LLMChain(llm=choicellm, prompt=multiple_choice_prompt, output_key="answer")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.1/789.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.6 MB/s eta 0:00:00


In [4]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
print(choice_chain(question))

{'question': '我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？', 'answer': 'A. 一个能够查询商品信息，为用户进行商品导购的工具。'}


In [5]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
print(choice_chain(question)) 

{'question': '我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？', 'answer': 'B. 一个能够查询订单信息，获得最新的订单情况的工具。'}


In [6]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
print(choice_chain(question))


{'question': '请问你们的货，能送到三亚吗？大概需要几天？', 'answer': 'C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具。'}


In [7]:
question = "今天天气怎么样？"
print(choice_chain(question))

{'question': '今天天气怎么样？', 'answer': 'D. 都不合适。该用户的问题与商品、订单、商家信息无关，需要使用天气预报工具或者查询天气应用来回答。'}


# Agent实现


In [8]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

def search_order(input: str) -> str:
    return "订单状态：已发货；发货日期：2023-01-01；预计送达时间：2023-01-10"

def recommend_product(input: str) -> str:
    return "红色连衣裙"

def faq(intput: str) -> str:
    return "7天无理由退货"

tools = [
    Tool(
        name = "Search Order",func=search_order, 
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(name="Recommend Product", func=recommend_product, 
         description="useful for when you need to answer questions about product recommendations"
    ),
    Tool(name="FAQ", func=faq,
         description="useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."
    )
]

agent = initialize_agent(tools, OpenAI(temperature=0), agent="zero-shot-react-description", verbose=True)

In [9]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I need to recommend a product.
Action: Recommend Product
Action Input: Clothing
Observation: 红色连衣裙
Thought: I now know the final answer.
Final Answer: 我推荐红色连衣裙。

> Finished chain.
我推荐红色连衣裙。


In [10]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I need to find out the status of the order
Action: Search Order
Action Input: 2022ABCDE
Observation: 订单状态：已发货；发货日期：2023-01-01；预计送达时间：2023-01-10
Thought: I now know the final answer
Final Answer: 您的订单 2022ABCDE 已发货，预计将于2023-01-10送达。

> Finished chain.
您的订单 2022ABCDE 已发货，预计将于2023-01-10送达。


In [11]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I need to find out the shipping policy and delivery time
Action: FAQ
Action Input: Shipping policy and delivery time
Observation: 7天无理由退货
Thought: I need to find out the delivery time
Action: FAQ
Action Input: Delivery time
Observation: 7天无理由退货
Thought: I need to find out if we can deliver to Sanya
Action: FAQ
Action Input: Delivery to Sanya
Observation: 7天无理由退货
Thought: I now know the final answer
Final Answer: 我们可以把货送到三亚，大概需要7天。

> Finished chain.
我们可以把货送到三亚，大概需要7天。


# 通过max_iterations限制重试次数


In [12]:
agent = initialize_agent(tools, OpenAI(temperature=0), agent="zero-shot-react-description", max_iterations = 2, verbose=True)
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print("===")
print(result)
print("===")



> Entering new AgentExecutor chain...
 I need to find out the shipping policy and delivery time
Action: FAQ
Action Input: Shipping policy and delivery time
Observation: 7天无理由退货
Thought: I need to find out the delivery time
Action: FAQ
Action Input: Delivery time
Observation: 7天无理由退货
Thought:
> Finished chain.
===
Agent stopped due to iteration limit or time limit.
===


# 通过VectorDBQA让Tool支持问答


In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain import OpenAI, VectorDBQA

from langchain.document_loaders import GoogleDriveLoader


llm = OpenAI(temperature=0)
loader = GoogleDriveLoader(
    folder_id="1OWx08kZrzsAh7LmBmtuwHo8teOnbr9pW",
    # Optional: configure whether to recursively fetch files from subfolders. Defaults to False.
    recursive=False
)
documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)

FileNotFoundError: ignored